## __PROJET 5 - IMDB Movie Prediction__ - _*Ludovic & Yasemin*_

####  __SECONDE PARTIE__ : MACHINE LEARNING

In [2]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
pd.set_option("display.max_columns", None)

df = pd.read_csv("./Datas/5000_movies_bis_clean.csv")
df.head(3)

,Unnamed: 0,duration,director_fb_likes,actor_1_fb_likes,gross,num_voted_users,facenumber_in_poster,budget,title_year,aspect_ratio,movie_fb_likes,country_UK,country_USA,other_actors_fb_likes,critic_reviews_ratio,imdb_classification
0,0,178.0,0.0,1000.0,760505847.0,886204,0.0,237000000.0,2009.0,1.78,33000,False,True,1791.0,0.236739,2
1,1,169.0,563.0,40000.0,309404152.0,471220,0.0,300000000.0,2007.0,2.35,0,False,True,6000.0,0.243942,2
2,2,148.0,0.0,11000.0,200074175.0,275868,1.0,245000000.0,2015.0,2.35,85000,True,False,554.0,0.605634,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4703 entries, 0 to 4702
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             4703 non-null   int64  
 1   duration               4703 non-null   float64
 2   director_fb_likes      4703 non-null   float64
 3   actor_1_fb_likes       4703 non-null   float64
 4   gross                  4703 non-null   float64
 5   num_voted_users        4703 non-null   int64  
 6   facenumber_in_poster   4703 non-null   float64
 7   budget                 4703 non-null   float64
 8   title_year             4703 non-null   float64
 9   aspect_ratio           4703 non-null   float64
 10  movie_fb_likes         4703 non-null   int64  
 11  country_UK             4703 non-null   bool   
 12  country_USA            4703 non-null   bool   
 13  other_actors_fb_likes  4703 non-null   float64
 14  critic_reviews_ratio   4703 non-null   float64
 15  imdb